In [9]:
import pandas as pd

# Caminho do arquivo Excel com múltiplas planilhas
arquivo = r"C:\Users\Marco\Documents\LEADR\Dados - Leadr\estabelecimentos.xlsx"

# Carregar as duas planilhas
df1 = pd.read_excel(arquivo, sheet_name="Municípios")
df2 = pd.read_excel(arquivo, sheet_name="Regiões de Planejamento")

# Função para formatar valores numéricos
def format_num(x):
    if pd.isna(x):
        return ""
    elif isinstance(x, (int, float)) and float(x) % 1 == 0:
        return f"{int(x)}"
    elif isinstance(x, (int, float)):
        return f"{x:.2f}".replace('.', ',')
    else:
        return x

# Detectar colunas numéricas antes da formatação para passar para a função
num_cols_df1 = df1.select_dtypes(include=['float64', 'int64']).columns.tolist()
num_cols_df2 = df2.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Aplicar a formatação nas colunas numéricas
for df, num_cols in zip([df1, df2], [num_cols_df1, num_cols_df2]):
    for col in num_cols:
        df[col] = df[col].apply(format_num)

# Função para gerar HTML da tabela com classes para alinhamento numérico no <th> e <td>
def df_to_html_with_num_align(df, table_id, num_cols):
    html = f'<table id="{table_id}" class="display dataTable" cellspacing="0" width="100%">'
    # Cabeçalho com classe 'num' para colunas numéricas
    html += '<thead><tr>'
    for col in df.columns:
        if col in num_cols:
            html += f'<th class="num">{col}</th>'
        else:
            html += f'<th>{col}</th>'
    html += '</tr></thead>'
    # Corpo da tabela
    html += '<tbody>'
    for _, row in df.iterrows():
        html += '<tr>'
        for col in df.columns:
            cell_value = row[col]
            if col in num_cols:
                html += f'<td class="num">{cell_value}</td>'
            else:
                html += f'<td>{cell_value}</td>'
        html += '</tr>'
    html += '</tbody></table>'
    return html

# Gerar HTML para as duas tabelas com alternância
html_string = f"""
<html>
<head>
  <meta charset="utf-8" />
  <title>Estabelecimentos Agropecuários do Ceará - 2017</title>
  <link rel="stylesheet" href="https://cdn.datatables.net/1.13.4/css/jquery.dataTables.min.css">
  <style>
    body {{
      font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
      margin: 40px;
      background-color: #fafafa;
      color: #222;
      position: relative;
      min-height: 100vh;
      padding-bottom: 70px;
    }}

    .tab {{
      display: none;
    }}

    .tab-buttons {{
      margin-bottom: 20px;
    }}

    .tab-buttons button {{
      background-color: #ddd;
      border: none;
      padding: 10px 20px;
      cursor: pointer;
      margin-right: 5px;
      font-weight: bold;
    }}

    .tab-buttons button.active {{
      background-color: #2980b9;
      color: white;
    }}

    table.dataTable {{
      border-collapse: separate !important;
      border-spacing: 0 6px;
      background-color: white;
      box-shadow: 0 1px 3px rgb(0 0 0 / 0.06);
      width: 100%;
    }}

    table.dataTable thead th {{
      background-color: transparent;
      color: #555;
      font-weight: 600;
      padding: 8px 15px;
      border: none !important;
    }}

    /* Forçar alinhamento à direita nas colunas numéricas (thead e tbody) */
    table.dataTable thead th.num,
    table.dataTable tbody td.num {{
      text-align: right !important;
    }}

    /* Alinhamento à esquerda nas demais colunas */
    table.dataTable thead th:not(.num),
    table.dataTable tbody td:not(.num) {{
      text-align: left !important;
    }}

    table.dataTable tbody td {{
      padding: 8px 15px;
      border: none !important;
      vertical-align: middle;
    }}

    table.dataTable tbody tr {{
      background-color: #fff;
      box-shadow: 0 0.5px 2px rgb(0 0 0 / 0.04);
      margin-bottom: 6px;
      display: table-row;
    }}

    table.dataTable tbody tr:hover {{
      background-color: #f0f4ff;
    }}

    div.dataTables_wrapper .dataTables_filter {{
      font-weight: 600;
      color: #555;
      margin-bottom: 10px;
    }}

    div.dataTables_wrapper .dataTables_filter input {{
      border: 1px solid #ccc;
      padding: 6px 12px;
      border-radius: 0 !important;
      margin-left: 0.5em;
      font-size: 14px;
      width: 250px;
    }}

    div.dataTables_wrapper .dataTables_paginate,
    div.dataTables_wrapper .dataTables_info,
    div.dataTables_wrapper .dataTables_length {{
      display: none;
    }}

    footer {{
      position: fixed;
      bottom: 0;
      left: 0;
      width: 100%;
      height: 36px;
      background-color: #f0f0f0;
      color: #555;
      font-size: 13px;
      line-height: 36px;
      text-align: center;
      border-top: 1px solid #ddd;
      z-index: 9999;
    }}

    #download-excel {{
      position: fixed;
      bottom: 44px;
      right: 20px;
      padding: 6px 12px;
      font-size: 12px;
      border-radius: 0;
      cursor: pointer;
      background-color: #2980b9;
      color: white;
      border: none;
      opacity: 0.75;
      transition: opacity 0.3s ease;
      z-index: 10000;
    }}
    #download-excel:hover {{
      opacity: 1;
      background-color: #1f5d85;
    }}
  </style>
  <script src="https://code.jquery.com/jquery-3.5.1.js"></script>
  <script src="https://cdn.datatables.net/1.13.4/js/jquery.dataTables.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/xlsx@0.18.5/dist/xlsx.full.min.js"></script>
</head>
<body>

<h2>Estabelecimentos Agropecuários do Ceará - 2017</h2>

<div class="tab-buttons">
  <button onclick="showTab('tab1', this)" class="active">Municípios</button>
  <button onclick="showTab('tab2', this)">Regiões de Planejamento</button>
</div>

<div id="tab1" class="tab" style="display:block;">
  {df_to_html_with_num_align(df1, "table1", num_cols_df1)}
</div>

<div id="tab2" class="tab">
  {df_to_html_with_num_align(df2, "table2", num_cols_df2)}
</div>

<button id="download-excel" title="Baixar dados Excel">Baixar dados</button>

<footer>
  Fonte: Censo Agropecuário - IBGE, 2017
</footer>

<script>
function showTab(tabId, btn) {{
  $('.tab').hide();
  $('#' + tabId).show();
  $('.tab-buttons button').removeClass('active');
  $(btn).addClass('active');
}}

$(document).ready(function() {{
  $('#table1').DataTable({{
    paging: false,
    searching: true,
    ordering: true,
    info: false,
    lengthChange: false,
    language: {{
      search: "Pesquisar:"
    }}
  }});

  $('#table2').DataTable({{
    paging: false,
    searching: true,
    ordering: true,
    info: false,
    lengthChange: false,
    language: {{
      search: "Pesquisar:"
    }}
  }});

  document.getElementById('download-excel').addEventListener('click', function () {{
    var wb = XLSX.utils.book_new();
    var ws1 = XLSX.utils.table_to_sheet(document.getElementById('table1'));
    var ws2 = XLSX.utils.table_to_sheet(document.getElementById('table2'));
    XLSX.utils.book_append_sheet(wb, ws1, "Municípios");
    XLSX.utils.book_append_sheet(wb, ws2, "Regiões");
    XLSX.writeFile(wb, 'estabelecimentos.xlsx');
  }});
}});
</script>

</body>
</html>
"""

# Caminho para salvar o arquivo HTML
caminho_html = r"C:\Users\Marco\Documents\LEADR\Dados - Leadr\tabela_estabelecimentos_ceara_2017.html"
with open(caminho_html, "w", encoding="utf-8") as f:
    f.write(html_string)

print("Arquivo HTML gerado em:", caminho_html)


Arquivo HTML gerado em: C:\Users\Marco\Documents\LEADR\Dados - Leadr\tabela_estabelecimentos_ceara_2017.html
